WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx  
hdro_indicators_aggregates_sdn.csv  
hdro_indicators_sdn.csv  
indicators_sdn.csv  
sdg_indicatorlist_sdn.csv  
sdg_data_sdn.csv  
gender_sdn.csv  

In [109]:
import pandas as pd
import numpy as np
import re

In [110]:
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [111]:
!ls ./MyDrive/MyDrive/omneda_sudan/

dtm_sdn_smu-bi-weekly-13-_-17122024_v02_public_hdx.xlsx
education_sdn.csv
gender_sdn.csv
hdro_indicators_aggregates_sdn.csv
hdro_indicators_sdn.csv
indicators_sdn.csv
primary-secondary-enrollment-completion-rates.csv
sdg_data_sdn.csv
sdg_indicatorlist_sdn.csv
WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx
WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS.xlsx


### Read in the United Nations demographic indicator data

###### WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx

In [112]:
demo_indicator = pd.read_excel('./MyDrive/MyDrive/omneda_sudan/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx',
                               sheet_name="Estimates",
                               skiprows=15, header=[0,1])

In [113]:
demo_indicator.head()

Unnamed: 0_level_0 Unnamed: 1_level_0                   Unnamed: 2_level_0  \
               Index            Variant Region, subregion, country or area *   
0                  1          Estimates                                World   
1                  2          Estimates                                World   
2                  3          Estimates                                World   
3                  4          Estimates                                World   
4                  5          Estimates                                World   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
               Notes      Location code    ISO3 Alpha-code    ISO2 Alpha-code   
0                NaN                900                NaN                NaN   
1                NaN                900                NaN                NaN   
2                NaN                900                NaN                NaN   
3                NaN                900                NaN                NaN   
4                NaN                900                NaN                NaN   

  Unnamed: 7_level_0 Unnamed: 8_level_0 Unnamed: 9_level_0  ...  \
         SDMX code**               Type        Parent code  ...   
0                1.0              World                  0  ...   
1                1.0              World                  0  ...   
2                1.0              World                  0  ...   
3                1.0              World                  0  ...   
4                1.0              World                  0  ...   

                                                                      Mortality  \
  Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)   
0                                              580.5                              
1                                            566.566                              
2                                            546.444                              
3                                            535.811                              
4                                            522.058                              

                                                                                     \
  Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)   
0                                            497.388                                  
1                                            488.435                                  
2                                             475.37                                  
3                                            467.361                                  
4                                            455.621                                  

                                                                                               \
  Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)   
0                                            238.516                                            
1                                            229.703                                            
2                                            217.311                                            
3                                            211.257                                            
4                                            203.337                                            

                                                                                              \
  Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)   
0                                            268.734                                           
1                                            256.236                                           
2                                             238.56                                           
3                                            230.96

In [114]:
# demo_indicator.columns.values
demo_indicator.columns=['_'.join(col).strip() for col in demo_indicator.columns.values]
demo_indicator.columns=[re.findall("level_0_(.*)", col)[0]  if col.startswith('Unnamed') else col for col in demo_indicator.columns ]
demographics=demo_indicator[demo_indicator['Region, subregion, country or area *'].eq('Sudan')]

# demo_indicator.columns

In [115]:
def mergeIntoDemographics(df, colName='Indicator Name', indexName='Year', valueName='Value', suffix=None ):

  return demographics.merge(df.pivot(columns=colName, values = valueName, index=indexName).reset_index(),
                                 left_on  = 'Year',
                                 right_on = indexName,
                                 how      = 'left',
                                 suffixes=(None, suffix)
                                 )

def cleanFile(df, yearCol='Year', indicatorCol='Indicator Name'):
  df.drop(0, inplace=True)
  df[yearCol]      = df[yearCol].apply(pd.to_numeric)
  df[indicatorCol] = df[indicatorCol].astype(str)
  df.drop_duplicates(subset=[yearCol, indicatorCol], inplace=True)
  return df


In [116]:
len(demographics.columns)

65

In [117]:
df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_aggregates_sdn.csv')
yearCol = 'year'
indicatorCol = 'indicator_name'
valCol = 'value'
df=cleanFile(df, yearCol, indicatorCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'hdro')

In [118]:
len(demographics.columns)

71

In [119]:
df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_sdn.csv')
yearCol = 'year'
indicatorCol = 'indicator_name'
valCol = 'value'
df=cleanFile(df, yearCol, indicatorCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'hdro')
print(f"demographics column count {len(demographics.columns)}")

demographics column count 117


In [120]:
df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/indicators_sdn.csv')
yearCol = 'Year'
indicatorCol = 'Indicator Name'
valCol = 'Value'
df=cleanFile(df, yearCol, indicatorCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'indicators')
print(f"demographics column count {len(demographics.columns)}")



demographics column count 3000


In [121]:
sdg      = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_indicatorlist_sdn.csv')
sdg_data = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_data_sdn.csv')
df       = sdg_data.merge(sdg, on='indicator_id', how='left')


yearCol = 'year'
indicatorCol = 'indicator_label_en'
valCol = 'value'

df=cleanFile(df, yearCol, indicatorCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'sdg')
print(f"demographics column count {len(demographics.columns)}")




demographics column count 4051


In [122]:
df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/gender_sdn.csv')
yearCol = 'Year'
indicatorCol = 'Indicator Name'
valCol = 'Value'
df=cleanFile(df, yearCol, indicatorCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'gender')
print(f"demographics column count {len(demographics.columns)}")


demographics column count 4271


In [123]:
df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/education_sdn.csv')
yearCol = 'Year'
indicatorCol = 'Indicator Name'
valCol = 'Value'
df=cleanFile(df, yearCol, indicatorCol)
demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, 'education')
print(f"demographics column count {len(demographics.columns)}")



demographics column count 5055


In [124]:
demographics.to_csv('demographics.csv')

In [125]:
# df = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/primary-secondary-enrollment-completion-rates.csv')

In [126]:
# df[df['Entity'] =='Sudan']

### Read Aggregated Human Development Indicators for Sudan
hdro_indicators_aggregates_sdn.csv

In [127]:
# hdro_indicators_agg=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_aggregates_sdn.csv')
# hdro_indicators_agg.drop(0, inplace=True)
# hdro_indicators_agg['year'] = hdro_indicators_agg['year'].apply(pd.to_numeric)

# hdro_indicators_agg.head()

In [128]:
# hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index().head()

### merge into demographics

In [129]:
# demographics= demographics.merge(hdro_indicators_agg.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='year',
#                                  how='left'
#                                  ).drop(columns=('year'))

In [130]:
# demographics.columns

#### hdro_indicators_sdn.csv

In [131]:
# hdro_indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_sdn.csv')
# hdro_indicators.drop(0, inplace=True)
# hdro_indicators['year'] = hdro_indicators['year'].apply(pd.to_numeric)

# hdro_indicators.head()

In [132]:
# demographics= demographics.merge(hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='year',
#                                  how='left'
#                                  ).drop(columns=('year'))

In [133]:
# hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index().head()

In [134]:
# hdro_indicators.columns

In [135]:
# hdro_indicators.iloc[25:].head(10)

In [136]:
# hdro_indicators['index_indicator_name']=hdro_indicators['index_name'] + "_" + hdro_indicators['indicator_name']

In [137]:
# hdro_indicators.pivot(columns=  ['index_indicator_name'], values = 'value', index=['year']).reset_index().head()

In [138]:
# demographics.head().to_csv('demographics_top5.csv')

#### Indicators SDN
hdro_indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_sdn.csv')
hdro_indicators.head()

In [139]:
# indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/indicators_sdn.csv')
# indicators.drop(0, inplace=True)
# indicators.drop_duplicates(subset=['Year', 'Indicator Name'], inplace=True)
# # indicators.drop_duplicates(subset=['Year', 'Indicator Name', 'Value'], inplace=True)
# indicators['Year'] = indicators['Year'].apply(pd.to_numeric)
# indicators.head()

In [140]:
# indicators.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index()

In [141]:
# demographics= demographics.merge(indicators.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='Year',
#                                  how='left'
#                                  )

#### SDG Indicator list and data

In [142]:
# sdg=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_indicatorlist_sdn.csv')
# sdg_data=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_data_sdn.csv')
# sdg.iloc[10:].head()

In [143]:

# sdg_data.head()

In [144]:
# sdg_merge=sdg_data.merge(sdg, on='indicator_id', how='left')
# sdg_merge.drop(0, inplace=True)
# sdg_merge['year']=sdg_merge['year'].apply(pd.to_numeric)
# sdg_merge['indicator_label_en']=sdg_merge['indicator_label_en'].astype(str)
# sdg_merge.drop_duplicates(subset=['year', 'indicator_label_en'], inplace=True)
# sdg_merge.head()


In [145]:
# sdg_merge.pivot(columns=['indicator_label_en'], values = 'value', index=['year']).reset_index().head()

In [146]:
# demographics= demographics.merge(sdg_merge.pivot(columns=['indicator_label_en'], values = 'value', index=['year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='year',
#                                  how='left'
#                                  ).drop(columns='year')

In [147]:
# gender = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/gender_sdn.csv')
# gender.drop(0, inplace=True)
# gender['Indicator Name']=gender['Indicator Name'].astype(str)
# gender['Year'] = indicators['Year'].apply(pd.to_numeric)
# gender.drop_duplicates(subset=['Year', 'Indicator Name'], inplace=True)
# gender.head()

In [148]:
# gender.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index().head()

In [149]:
# demographics= demographics.merge(gender.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index(),
#                                  left_on='Year',
#                                  right_on='Year',
#                                  how='left'
#                                  )
# # .drop(columns='year')

In [150]:
# df=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/education_sdn.csv')
# df=cleanFile(df, 'Year', 'Indicator Name')
# demographics = mergeIntoDemographics(df, 'Indicator Name', 'Year', 'Value', 'education')

In [151]:
# education = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/education_sdn.csv')
# education.drop(0, inplace=True)
# education['Indicator Name']=gender['Indicator Name'].astype(str)
# education['Year'] = indicators['Year'].apply(pd.to_numeric)
# gender.drop_duplicates(subset=['Year', 'Indicator Name'], inplace=True)
# education.head()

In [152]:
demographics= demographics.merge(education.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index(),
                                 left_on='Year',
                                 right_on='Year',
                                 how='left'
                                 )
# .drop(columns='year')

ValueError: You are trying to merge on float64 and object columns for key 'Year'. If you wish to proceed you should use pd.concat

In [ ]:
# demographics.head()

In [ ]:
# demographics.to_csv('demographics.csv')